In [56]:
# 🛠️ Step 1: Install dependencies
!pip install streamlit colabcode matplotlib numpy -q
# !npm install -g localtunnel
!pip install --upgrade pyngrok


In [57]:
# 🧠 Step 2: Write tsp_ga.py
%%writefile tsp_ga.py
import random
import numpy as np

def total_distance(route, dist_matrix):
    return sum(dist_matrix[route[i-1]][route[i]] for i in range(len(route)))

def create_initial_population(n_cities, pop_size):
    return [random.sample(range(n_cities), n_cities) for _ in range(pop_size)]

def mutate(route):
    a, b = random.sample(range(len(route)), 2)
    route[a], route[b] = route[b], route[a]
    return route

def crossover(p1, p2):
    cut = random.randint(0, len(p1) - 1)
    child = p1[:cut] + [c for c in p2 if c not in p1[:cut]]
    return child

def evolve_population(pop, dist_matrix, elite_size):
    scores = [(r, total_distance(r, dist_matrix)) for r in pop]
    scores.sort(key=lambda x: x[1])
    elite = [r for r, _ in scores[:elite_size]]
    next_gen = elite[:]
    while len(next_gen) < len(pop):
        parent1, parent2 = random.sample(elite, 2)
        child = crossover(parent1, parent2)
        if random.random() < 0.2:
            child = mutate(child)
        next_gen.append(child)
    return next_gen, scores[0][0], scores[0][1]

Overwriting tsp_ga.py


In [58]:
# 📦 Step 3: Write utils.py
%%writefile utils.py
import numpy as np
import matplotlib.pyplot as plt

def generate_cities(n):
    return np.random.rand(n, 2) * 100

def compute_distance_matrix(cities):
    n = len(cities)
    return [[np.linalg.norm(cities[i] - cities[j]) for j in range(n)] for i in range(n)]

def plot_route(cities, route):
    fig, ax = plt.subplots()
    ordered = cities[route + [route[0]]] if isinstance(route, list) else cities[np.append(route, route[0])]
    ax.plot(ordered[:, 0], ordered[:, 1], 'o-', color='blue')
    for i, (x, y) in enumerate(cities):
        ax.text(x + 1, y + 1, str(i), fontsize=8)
    ax.set_title("Optimized Route")
    return fig

Overwriting utils.py


In [59]:
# 🚀 Step 4: Write app.py (Streamlit app)
%%writefile app.py
import streamlit as st
from tsp_ga import *
from utils import *
import time

st.title("🚚 Route Optimization with Genetic Algorithm")
n_cities = st.slider("Number of cities", 5, 20, 10)
generations = st.slider("Generations", 10, 500, 100)
pop_size = 100
elite_size = 20

if st.button("Generate and Optimize Route"):
    cities = generate_cities(n_cities)
    dist_matrix = compute_distance_matrix(cities)
    pop = create_initial_population(n_cities, pop_size)

    best_distances = []
    best_route = []
    for _ in range(generations):
        pop, best_route, best_distance = evolve_population(pop, dist_matrix, elite_size)
        best_distances.append(best_distance)

    st.success(f"Best Distance: {best_distance:.2f}")
    st.pyplot(plot_route(cities, best_route))
    st.line_chart(best_distances)

Overwriting app.py


In [60]:
# 🔓 Step 5: Run the Streamlit app using colabcode

from pyngrok import conf
conf.get_default().auth_token = "2qFLPrbBC3ehO7sBa8BBJ2hJDu0_7PYyEj4KBfyVEJswmGtT"
from colabcode import ColabCode
ColabCode(port=8501)

# !streamlit run app.py &>/content/log.txt &
# !npx localtunnel --port 8500


ERROR:  authentication failed: Usage of ngrok requires a verified account and authtoken.
ERROR:  
ERROR:  Sign up for an account: https://dashboard.ngrok.com/signup
ERROR:  Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_4018
ERROR:  https://ngrok.com/docs/errors/err_ngrok_4018
ERROR:  
